In [1]:
import medusa
import pandas as pd

from pickle import load

from medusa.flux_analysis.deletion import ensemble_single_gene_deletion

from cobra.flux_analysis.deletion import single_gene_deletion

In [2]:
# load the previously generated ensemble
with open('../results/ensembles/psy_ensemble_500_SEED_biomass_round2.pickle', 'rb') as infile:
    ensemble = load(infile)

In [3]:
ensemble.base_model.objective.expression

1.0*bio1 - 1.0*bio1_reverse_b18f7

In [4]:
# make sure all exchange reactions are open to calculate
# growth rate in complete media
ex_rxns = [rxn for rxn in ensemble.base_model.reactions if
              rxn.id.startswith('EX_')]
for rxn in ex_rxns:
    rxn.lower_bound = -1000

# identify genes that are essential in all ensemble members by opening 
# all reactions and testing for essentiality

# Open all reactions in the ensemble
for feature in ensemble.features:
    component = ensemble.base_model.reactions.get_by_id(feature.base_component.id)
    component.lower_bound = -1000
    component.upper_bound = 1000

# Perform the essentiality simulations with all exchanges open in the base model
always_essential = single_gene_deletion(ensemble.base_model)

# Filter by always being essential
always_essential = always_essential.loc[always_essential['growth'] < 1E-6]
always_essential.index = [list(gene)[0] for gene in always_essential.index]
always_essential_genes = [ensemble.base_model.genes.get_by_id(g) 
                          for g in always_essential.index]
genes_to_ko = [g for g in ensemble.base_model.genes if g not in always_essential_genes]

# Perform the knockouts for genes that weren't essential in the opened base model
gene_knockout_results = ensemble_single_gene_deletion(ensemble, specific_genes = genes_to_ko)
knockout_df = pd.DataFrame()
for member in gene_knockout_results:
    gene_knockout_results[member].index = [list(gene)[0] for gene in gene_knockout_results[member].index]
    knockout_df[member] = gene_knockout_results[member]['growth']

knockout_df.to_csv('../results/ensemble_gene_knockouts_round2.tsv', sep = '\t')

performing deletions for PST_gapfilled_22
performing deletions for PST_gapfilled_398
performing deletions for PST_gapfilled_122
performing deletions for PST_gapfilled_63
performing deletions for PST_gapfilled_255
performing deletions for PST_gapfilled_80
performing deletions for PST_gapfilled_434
performing deletions for PST_gapfilled_437
performing deletions for PST_gapfilled_246
performing deletions for PST_gapfilled_333
performing deletions for PST_gapfilled_459
performing deletions for PST_gapfilled_177
performing deletions for PST_gapfilled_52
performing deletions for PST_gapfilled_186
performing deletions for PST_gapfilled_93
performing deletions for PST_gapfilled_481
performing deletions for PST_gapfilled_140
performing deletions for PST_gapfilled_225
performing deletions for PST_gapfilled_103
performing deletions for PST_gapfilled_232
performing deletions for PST_gapfilled_204
performing deletions for PST_gapfilled_107
performing deletions for PST_gapfilled_126
performing delet